In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

class ConvolutionalModel(tf.keras.Model):
    
    def __init__(self, num_classes):
        super(ConvolutionalModel, self).__init__()
        self.conv1 = layers.Conv2D(16, (5, 5), strides=(1, 1), padding="same", input_shape=(150, 150, 1))
        self.relu1 = layers.Activation('relu')
        self.maxpool1 = layers.MaxPooling2D((3, 3), strides=(2,2))

        self.conv2 = layers.Conv2D(16, (5, 5), strides=(1, 1), padding="same")
        self.relu2 = layers.Activation('relu')
        self.maxpool2 = layers.MaxPooling2D((3, 3))
        
        self.conv3 = layers.Conv2D(16, (5, 5), strides=(1, 1), padding="same")
        self.relu3 = layers.Activation('relu')
        self.maxpool3 = layers.MaxPooling2D((3, 3))

        self.conv4 = layers.Conv2D(16, (9, 9), strides=(1, 1), padding="same")
        self.relu4 = layers.Activation('relu')
        self.maxpool4 = layers.MaxPooling2D((3, 3))
        
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False): 
        output_conv1 = self.conv1(inputs)
        output_relu1 = self.relu1(output_conv1)
        output_maxpool1 = self.maxpool1(output_relu1)

        output_conv2 = self.conv2(output_maxpool1)
        output_relu2 = self.relu2(output_conv2)
        output_maxpool2 = self.maxpool2(output_relu2)

        output_conv3 = self.conv3(output_maxpool2)
        output_relu3 = self.relu3(output_conv3)
        output_maxpool3 = self.maxpool3(output_relu3)
        
        output_conv4 = self.conv4(output_maxpool3)
        output_relu4 = self.relu4(output_conv4)
        output_maxpool4 = self.maxpool4(output_relu4)
        
        output_flatten = self.flatten(output_maxpool4)
        output_dense = self.dense(output_flatten)
        return output_dense
